In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

# 6. Preprocess Data and Create Labels and Features

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/13O-5I0-hh__8n53OQF4TCDkEMeEm33y98EutsDE1yfE/export?format=csv")

actions =  np.array(list(df["actions_label"]))
label_map = {label:num for num, label in enumerate(actions)}

DATA_PATH = os.path.join('MainDataSet') 
# DATA_PATH = os.path.join('MP_Data') 
# Videos are going to be 30 frames in length
sequence_length = 30

In [4]:
actions

array(['empty', 'tumi_tumar_tomake',
       'ki_kothay_keno_kivabe_kokhn_koto_kemon', 'ami', 'amar', 'amader',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'hajar', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       'school', 'college', 'varsity', 'office', 'kaj', 'computer',
       'mobile', 'boi', 'kolom', 'light', 'tv', 'fan', 'cricket',
       'football', 'hata', 'kotha bola', 'ghumano',
       'asha_ashe_ashi_ashen_ashbo_ashben_ay', 'jawa', 'dekha',
       'din_dibosh', 'shoni bar', 'robi bar', 'shom bar', 'mongol bar',
       'budh bar', 'brihoshpoti bar', 'shukro bar', 'bivag', 'dhaka',
       'chattogram', 'khulna', 'rajshahi', 'barisal', 'sylhet', 'rangpur',
       'mymensingh', 'vaat', 'maach', 'mangsho_gushto', 'murgi', 'cha',
       'kofi', 'ruti', 'sobji', 'daal', 'mishti', 'paani', 'dudh', 'aam',
       'kathal', 'aapel', 'komola', 'kola', 'aangur', 'pepe', 'kha

In [5]:
sequences, labels = [], []
index = 0
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH,  f"{index}. {action}"))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, f"{index}. {action}", str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    index+=1

In [ ]:
sequences

In [6]:
np.array(sequences).shape

(8790, 30, 1662)

In [10]:
np.array(labels).shape

(6690,)

In [7]:
X = np.array(sequences)

In [17]:
X.shape

(4170, 30, 1662)

In [9]:
y = to_categorical(labels).astype(int)

In [19]:
y.shape

(4170, 52)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [14]:
# y_test.shape
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6355, 30, 1662)
(335, 30, 1662)
(6355, 80)
(335, 80)


# 7. Build and Train LSTM Neural Network

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard

In [12]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [27]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
model.output_shape

(None, 100)

In [239]:
res = [0.2,0.7,0.1]

In [240]:
res

[0.2, 0.7, 0.1]

In [241]:
actions[np.argmax(res)]

'tumi_tumar_tomake'

In [29]:
from keras.optimizers import Adam
# opt = Adam()
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [40]:
model.fit(X_train, y_train, epochs=120, callbacks=[tb_callback])

Epoch 1/120
261/261 [==============================] - 114s 419ms/step - loss: 0.1895 - categorical_accuracy: 0.9420
Epoch 2/120
261/261 [==============================] - 108s 413ms/step - loss: 0.1792 - categorical_accuracy: 0.9396
Epoch 3/120
261/261 [==============================] - 112s 428ms/step - loss: 0.1584 - categorical_accuracy: 0.9492
Epoch 4/120
261/261 [==============================] - 106s 408ms/step - loss: 0.1833 - categorical_accuracy: 0.9416
Epoch 5/120
261/261 [==============================] - 106s 407ms/step - loss: 0.2599 - categorical_accuracy: 0.9163
Epoch 6/120
261/261 [==============================] - 110s 421ms/step - loss: 0.1675 - categorical_accuracy: 0.9549
Epoch 7/120
130/261 [=============>................] - ETA: 55s - loss: 0.1491 - categorical_accuracy: 0.9579

KeyboardInterrupt: 

In [41]:
# accuracy
print(X_train.shape[0])
res = model.predict(X_train)
count=0
for i in range(0,X_train.shape[0]): 
    if actions[np.argmax(res[i])] == actions[np.argmax(y_train[i])]:
        count+=1
print(count*100/X_train.shape[0])

8350
261/261 [==============================] - 44s 140ms/step
96.16766467065868


# 9. Save Weights

In [47]:
model.save('finalmodelv2.h5')

In [98]:
del model

In [106]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [42]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [48]:
yhat = model.predict(X_test)
yhatTrain = model.predict(X_train)

261/261 [==============================] - 42s 136ms/step


In [49]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [50]:
ytrueTrain = np.argmax(y_train, axis=1).tolist()
yhatTrain = np.argmax(yhatTrain, axis=1).tolist()

In [51]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[436,   0],
        [  1,   3]],

       [[423,   4],
        [  3,  10]],

       [[430,   2],
        [  2,   6]],

       [[426,   4],
        [  2,   8]],

       [[431,   1],
        [  0,   8]],

       [[438,   0],
        [  0,   2]],

       [[438,   0],
        [  0,   2]],

       [[437,   0],
        [  3,   0]],

       [[434,   0],
        [  0,   6]],

       [[436,   1],
        [  2,   1]],

       [[430,   2],
        [  4,   4]],

       [[432,   1],
        [  1,   6]],

       [[436,   0],
        [  0,   4]],

       [[433,   1],
        [  1,   5]],

       [[434,   2],
        [  0,   4]],

       [[435,   1],
        [  0,   4]],

       [[435,   1],
        [  1,   3]],

       [[436,   0],
        [  0,   4]],

       [[436,   0],
        [  0,   4]],

       [[434,   0],
        [  0,   6]],

       [[431,   0],
        [  1,   8]],

       [[435,   1],
        [  1,   3]],

       [[429,   1],
        [  3,   7]],

       [[435,   1],
        [  0, 

In [52]:
multilabel_confusion_matrix(ytrueTrain, yhatTrain)

array([[[8261,    3],
        [   0,   86]],

       [[8148,    5],
        [  17,  180]],

       [[8080,    8],
        [   7,  255]],

       [[8137,   13],
        [  25,  175]],

       [[8116,   32],
        [   9,  193]],

       [[8322,    0],
        [   4,   24]],

       [[8256,    6],
        [  14,   74]],

       [[8261,    2],
        [  22,   65]],

       [[8252,   14],
        [   5,   79]],

       [[8260,    3],
        [  15,   72]],

       [[8254,   14],
        [  15,   67]],

       [[8248,   19],
        [   3,   80]],

       [[8260,    4],
        [   2,   84]],

       [[8260,    6],
        [  14,   70]],

       [[8249,   15],
        [   6,   80]],

       [[8259,    5],
        [   5,   81]],

       [[8263,    1],
        [   4,   82]],

       [[8263,    1],
        [   1,   85]],

       [[8261,    3],
        [   2,   84]],

       [[8263,    3],
        [   4,   80]],

       [[8269,    0],
        [   4,   77]],

       [[8259,    5],
        [  1

In [53]:
accuracy_score(ytrue, yhat)

0.9022727272727272

In [54]:
accuracy_score(ytrueTrain, yhatTrain)


0.9616766467065868